In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

import warnings; warnings.simplefilter('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
xls = pd.ExcelFile('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx')
xls.sheet_names

['Bremer', 'Liquid supply', 'Gama', 'Premier', 'anker']

In [4]:
df1 = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx', sheet_name=xls.sheet_names[0])
df1 = df1.dropna(thresh=4)
df1 = df1.drop(['Your order\n qty./btls.'], axis=1)
print(df1.info())
df1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1837 entries, 2 to 1854
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Art. No.       1837 non-null   object 
 1   btl/cs         1837 non-null   float64
 2   cont.          1837 non-null   float64
 3   Vol %          1836 non-null   float64
 4   Description    1837 non-null   object 
 5   Unnamed: 5     1837 non-null   object 
 6   price/btl.     1837 non-null   object 
 7   price/cs.      1837 non-null   object 
 8   group          1837 non-null   object 
 9   btls/layer     1837 non-null   float64
 10  btls/pal       1837 non-null   float64
 11  EAN-Code/btl.  1825 non-null   float64
 12  Unnamed: 12    186 non-null    object 
dtypes: float64(6), object(7)
memory usage: 200.9+ KB
None


,Art. No.,btl/cs,cont.,Vol %,Description,Unnamed: 5,price/btl.,price/cs.,group,btls/layer,btls/pal,EAN-Code/btl.,Unnamed: 12
2,BEAPSET25,1.0,0.48,18.0,Berentzen Apfel Minis,Likör,6.7,6.7,LIKÖRE6 BZ,54.0,378.0,4.041500e+12,NaN
3,BEWALSET25,1.0,0.48,16.0,Berentzen Waldfrucht Minis,Likör,6.7,6.7,LIKÖRE6 BZ,54.0,378.0,4.041500e+12,NaN
4,HEGIFL,6.0,0.70,43.4,Hendricks Flora Adora Gin *LIMITED*,Gin aus Schottland,24.99,149.94,GIN,96.0,576.0,5.010328e+12,NaN
5,PUDIRO,6.0,0.70,18.0,Pure Divine Red Orange,französischer Likör,2.1,12.6,LIKÖRE FR,120.0,480.0,3.263280e+12,NaN
6,TESIETC10,6.0,1.00,18.0,Sierra Tropical Chilli,Passionfruit & Mango & Chilli,10.3,61.8,TEQUILA,108.0,540.0,4.062400e+12,NaN


In [5]:
df1.rename(columns = {'Description':'product', 'cont.':'volume', 'Vol %':'alcohol (%)', 'btl/cs':'bottles/case', 'price/btl.':'price/bottle', 'price/cs.':'price/case', 'Unnamed: 12':'offer' }, inplace = True)
df1['offer'] = df1['offer'].fillna('regular')
df1['volume'] = df1['volume'].apply(lambda x: x*100)
df1['vendor'] = xls.sheet_names[0]
df1['unit'] = 'CL'
df1.head()

,Art. No.,bottles/case,volume,alcohol (%),product,Unnamed: 5,price/bottle,price/case,group,btls/layer,btls/pal,EAN-Code/btl.,offer,vendor,unit
2,BEAPSET25,1.0,48.0,18.0,Berentzen Apfel Minis,Likör,6.7,6.7,LIKÖRE6 BZ,54.0,378.0,4.041500e+12,regular,Bremer,CL
3,BEWALSET25,1.0,48.0,16.0,Berentzen Waldfrucht Minis,Likör,6.7,6.7,LIKÖRE6 BZ,54.0,378.0,4.041500e+12,regular,Bremer,CL
4,HEGIFL,6.0,70.0,43.4,Hendricks Flora Adora Gin *LIMITED*,Gin aus Schottland,24.99,149.94,GIN,96.0,576.0,5.010328e+12,regular,Bremer,CL
5,PUDIRO,6.0,70.0,18.0,Pure Divine Red Orange,französischer Likör,2.1,12.6,LIKÖRE FR,120.0,480.0,3.263280e+12,regular,Bremer,CL
6,TESIETC10,6.0,100.0,18.0,Sierra Tropical Chilli,Passionfruit & Mango & Chilli,10.3,61.8,TEQUILA,108.0,540.0,4.062400e+12,regular,Bremer,CL


In [6]:
df2 = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx', sheet_name=xls.sheet_names[1], skiprows=[0])
df2 = df2.dropna(thresh=4)
print(df2.info())
df2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 1 to 76
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CASES       76 non-null     object 
 1   PRODUCT     76 non-null     object 
 2   Unnamed: 2  54 non-null     object 
 3   Unnamed: 3  76 non-null     object 
 4   STATUS      76 non-null     object 
 5   CASE        76 non-null     float64
 6   CONTENT     76 non-null     float64
 7   ALC %       76 non-null     float64
 8   PRICE       76 non-null     object 
 9   Unnamed: 9  76 non-null     object 
 10  LOCATION    76 non-null     object 
 11  REMARK      73 non-null     object 
dtypes: float64(3), object(9)
memory usage: 7.7+ KB
None


,CASES,PRODUCT,Unnamed: 2,Unnamed: 3,STATUS,CASE,CONTENT,ALC %,PRICE,Unnamed: 9,LOCATION,REMARK
1,60,ABERFELDY 12YRS,GBX,REF,E-AD,6.0,70.0,40.0,126,21,LOENDERSLOOT,ON THE FLOOR
2,176,ABERLOUR 12YRS DOUBLE CASK MATURED,GBX,REF,E-AD,3.0,70.0,40.0,80,26.67,LOENDERSLOOT,ON THE FLOOR
3,48,ARRAN BARREL RESERVE,GBX,REF,E-AD,6.0,70.0,43.0,108,18,LOENDERSLOOT,ON THE FLOOR
4,19,BAKERS 7YRS,GBX,REF,E-AD,6.0,70.0,53.5,299,49.83,LOENDERSLOOT,ON THE FLOOR
5,7,BENRIACH 22YRS TRIPLE DISTILLED,GBX,REF,E-AD,4.0,70.0,46.8,376,94,LOENDERSLOOT,ON THE FLOOR


In [7]:
df2['offer'] = 'regular' 
df2.rename(columns = {'PRODUCT':'product', 'CONTENT':'volume', 'ALC %':'alcohol (%)', 'CASE':'bottles/case', 'Unnamed: 9':'price/bottle', 'PRICE':'price/case'}, inplace = True)
df2['vendor'] = xls.sheet_names[1]
df2['unit'] = 'CL'
df2.head()

,CASES,product,Unnamed: 2,Unnamed: 3,STATUS,bottles/case,volume,alcohol (%),price/case,price/bottle,LOCATION,REMARK,offer,vendor,unit
1,60,ABERFELDY 12YRS,GBX,REF,E-AD,6.0,70.0,40.0,126,21,LOENDERSLOOT,ON THE FLOOR,regular,Liquid supply,CL
2,176,ABERLOUR 12YRS DOUBLE CASK MATURED,GBX,REF,E-AD,3.0,70.0,40.0,80,26.67,LOENDERSLOOT,ON THE FLOOR,regular,Liquid supply,CL
3,48,ARRAN BARREL RESERVE,GBX,REF,E-AD,6.0,70.0,43.0,108,18,LOENDERSLOOT,ON THE FLOOR,regular,Liquid supply,CL
4,19,BAKERS 7YRS,GBX,REF,E-AD,6.0,70.0,53.5,299,49.83,LOENDERSLOOT,ON THE FLOOR,regular,Liquid supply,CL
5,7,BENRIACH 22YRS TRIPLE DISTILLED,GBX,REF,E-AD,4.0,70.0,46.8,376,94,LOENDERSLOOT,ON THE FLOOR,regular,Liquid supply,CL


In [8]:
df3 = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx', sheet_name=xls.sheet_names[2])
df3 = df3.dropna(thresh=4)
df3['unit'] = 'CL'
print(df3.info())
df3.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1780 entries, 0 to 1779
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nº CODE       1780 non-null   object 
 1   BRANDS        1780 non-null   object 
 2   BTL           1780 non-null   float64
 3   L             1780 non-null   float64
 4   ALC%          1780 non-null   float64
 5   CAP           1776 non-null   object 
 6   PRICE/CS      1780 non-null   float64
 7   PRICE/BOTTLE  1780 non-null   float64
 8   STATUS        1780 non-null   object 
 9   OBSERVATIONS  134 non-null    object 
 10  CATEGORY      1779 non-null   object 
 11  unit          1780 non-null   object 
dtypes: float64(5), object(7)
memory usage: 180.8+ KB
None


,Nº CODE,BRANDS,BTL,L,ALC%,CAP,PRICE/CS,PRICE/BOTTLE,STATUS,OBSERVATIONS,CATEGORY,unit
0,TEQ.000.199.902,3 Josés Tequila Reposado 100% Agave Azul,6.0,0.7,40.0,RF,49.99998,8.33333,ON STOCK,NaN,TEQUILA & MEZCAL,CL
1,VOD.000.199.933,42 Below,6.0,0.7,40.0,RF,48.00000,8.00000,ON STOCK,NaN,VODKA,CL
2,VOD.000.199.646,9 Mille,6.0,1.0,37.5,RF,49.00002,8.16667,ON STOCK,NaN,VODKA,CL
3,COG.000.200.225,A.E. Dor Collector,6.0,0.7,40.0,RF,162.00000,27.00000,ON STOCK,NaN,COGNAC & BRANDY,CL
4,VOD.000.199.707,A.H. Riise Pharmacy Liquorice Shot Hot,6.0,0.7,18.0,RF,64.99998,10.83333,ON STOCK,NaN,VODKA,CL


In [9]:
df3.rename(columns = {'BRANDS':'product', 'L':'volume', 'ALC%':'alcohol (%)', 'BTL':'bottles/case', 'PRICE/BOTTLE':'price/bottle', 'PRICE/CS':'price/case', 'OBSERVATIONS':'offer' }, inplace = True)
df3['offer'] = df3['offer'].fillna('regular')
df3['volume'] = df3['volume'].apply(lambda x: x*100)
df3['vendor'] = xls.sheet_names[2]
df3.head()

,Nº CODE,product,bottles/case,volume,alcohol (%),CAP,price/case,price/bottle,STATUS,offer,CATEGORY,unit,vendor
0,TEQ.000.199.902,3 Josés Tequila Reposado 100% Agave Azul,6.0,70.0,40.0,RF,49.99998,8.33333,ON STOCK,regular,TEQUILA & MEZCAL,CL,Gama
1,VOD.000.199.933,42 Below,6.0,70.0,40.0,RF,48.00000,8.00000,ON STOCK,regular,VODKA,CL,Gama
2,VOD.000.199.646,9 Mille,6.0,100.0,37.5,RF,49.00002,8.16667,ON STOCK,regular,VODKA,CL,Gama
3,COG.000.200.225,A.E. Dor Collector,6.0,70.0,40.0,RF,162.00000,27.00000,ON STOCK,regular,COGNAC & BRANDY,CL,Gama
4,VOD.000.199.707,A.H. Riise Pharmacy Liquorice Shot Hot,6.0,70.0,18.0,RF,64.99998,10.83333,ON STOCK,regular,VODKA,CL,Gama


In [10]:
df4 = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx', sheet_name=xls.sheet_names[3], skiprows=[0,1,2,3,5])
df4 = df4.dropna(thresh=6)
df4 = df4.drop(df4.loc[:, ' ':].columns, axis=1)
df4['unit'] = 'CL'
print(df4.info())
df4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Quantity   506 non-null    object 
 1   Cases      506 non-null    object 
 2   Product    506 non-null    object 
 3   bottle     506 non-null    object 
 4   content    506 non-null    object 
 5   alc.vol    506 non-null    object 
 6   details    506 non-null    object 
 7   Euro       506 non-null    object 
 8   Case       506 non-null    float64
 9   Bottle     506 non-null    object 
 10  Available  506 non-null    object 
 11  Bottle.1   499 non-null    object 
 12  unit       506 non-null    object 
dtypes: float64(1), object(12)
memory usage: 55.3+ KB
None


,Quantity,Cases,Product,bottle,content,alc.vol,details,Euro,Case,Bottle,Available,Bottle.1,unit
0,232,cases,Aalborg Jubilaeum,6,70,40,rf/cl,Euro,45.6,7.6,stock,5700359304907,CL
1,678,cases,Aalborg Taffel Aquavit,6,70,45,rf/cl,Euro,34.5,5.75,stock,5700359304907,CL
2,39,cases,Aberfeldy 16y,6,70,40,rf/cl/gb,Euro,281.0,46.833333,stock,5000277000579,CL
3,6,cases,Aberfeldy 16y,6,100,40,rf/cl/gb,Euro,324.0,54,stock,7640171031354,CL
4,14,cases,Aberfeldy 18y Cote Rotie Red Wine Cask Finish,6,70,43,rf/cl/gb,Euro,552.0,92,stock,7640171037851,CL


In [11]:
df4['offer'] = 'regular' 
df4.rename(columns = {'Product':'product', 'content':'volume', 'alc.vol':'alcohol (%)', 'bottle':'bottles/case', 'Bottle':'price/bottle', 'Case':'price/case'}, inplace = True)
df4['vendor'] = xls.sheet_names[3]
df4.head()

,Quantity,Cases,product,bottles/case,volume,alcohol (%),details,Euro,price/case,price/bottle,Available,Bottle.1,unit,offer,vendor
0,232,cases,Aalborg Jubilaeum,6,70,40,rf/cl,Euro,45.6,7.6,stock,5700359304907,CL,regular,Premier
1,678,cases,Aalborg Taffel Aquavit,6,70,45,rf/cl,Euro,34.5,5.75,stock,5700359304907,CL,regular,Premier
2,39,cases,Aberfeldy 16y,6,70,40,rf/cl/gb,Euro,281.0,46.833333,stock,5000277000579,CL,regular,Premier
3,6,cases,Aberfeldy 16y,6,100,40,rf/cl/gb,Euro,324.0,54,stock,7640171031354,CL,regular,Premier
4,14,cases,Aberfeldy 18y Cote Rotie Red Wine Cask Finish,6,70,43,rf/cl/gb,Euro,552.0,92,stock,7640171037851,CL,regular,Premier


In [12]:
df5 = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/Spirits Akay.xlsx', sheet_name=xls.sheet_names[4])
df5 = df5.dropna(thresh=4)
df5['size'] = df5.apply(lambda x: x['size']/10 if x['Content'] == 'ml' else x['size'], axis=1)
df5['Content'] = df5.apply(lambda x: 'CL' if x['Content'] == 'ml' else x['Content'], axis=1)
df5['size'] = df5.apply(lambda x: x['size']*100 if x['Content'] == 'liter' else x['size'], axis=1)
df5['Content'] = df5.apply(lambda x: 'CL' if x['Content'] == 'liter' else x['Content'], axis=1)
print(df5.info())
df5.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4897 entries, 0 to 4896
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Description             4897 non-null   object 
 1   Case size               4897 non-null   int64  
 2   size                    4897 non-null   float64
 3   Content                 4897 non-null   object 
 4   alc %                   4897 non-null   float64
 5   Currency                4897 non-null   object 
 6   Price case              4897 non-null   float64
 7   Price bottle            4897 non-null   float64
 8   Category                4897 non-null   object 
 9   Order quantity bottles  0 non-null      float64
 10  Comment/remark          841 non-null    object 
 11  COO                     4897 non-null   object 
 12  barcode bottle          4429 non-null   float64
 13  barcode Outercase       4429 non-null   object 
dtypes: float64(6), int64(1), object(7)
memor

,Description,Case size,size,Content,alc %,Currency,Price case,Price bottle,Category,Order quantity bottles,Comment/remark,COO,barcode bottle,barcode Outercase
0,1724 Tonic,24,20.0,CL,0.0,€,25.44,1.06,Tonics / Soda / Energydrinks,NaN,NaN,SPAIN,8.437010e+12,18437009805883
1,1800 Anejo,6,70.0,CL,38.0,€,109.50,18.25,Tequila,NaN,Special,MEXICO,5.029977e+12,29977821024
2,1800 Anejo NRF,6,70.0,CL,38.0,€,112.50,18.75,Tequila,NaN,NaN,MEXICO,5.029977e+12,5029977821024
3,1800 Cristalino Anejo + GB,6,70.0,CL,35.0,€,189.00,31.50,Tequila,NaN,NaN,MEXICO,7.501035e+12,17501035013480
4,1800 Reposado,6,70.0,CL,38.0,€,97.50,16.25,Tequila,NaN,NaN,MEXICO,7.501035e+12,17501035013221


In [13]:
df5.rename(columns = {'Description':'product', 'size':'volume', 'Content':'unit', 'alc %':'alcohol (%)', 'Case size':'bottles/case', 'Price bottle':'price/bottle', 'Price case':'price/case', 'Comment/remark':'offer'}, inplace = True)
df5['offer'] = df5['offer'].fillna('regular')
df5['vendor'] = xls.sheet_names[4]
df5.head()

,product,bottles/case,volume,unit,alcohol (%),Currency,price/case,price/bottle,Category,Order quantity bottles,offer,COO,barcode bottle,barcode Outercase,vendor
0,1724 Tonic,24,20.0,CL,0.0,€,25.44,1.06,Tonics / Soda / Energydrinks,NaN,regular,SPAIN,8.437010e+12,18437009805883,anker
1,1800 Anejo,6,70.0,CL,38.0,€,109.50,18.25,Tequila,NaN,Special,MEXICO,5.029977e+12,29977821024,anker
2,1800 Anejo NRF,6,70.0,CL,38.0,€,112.50,18.75,Tequila,NaN,regular,MEXICO,5.029977e+12,5029977821024,anker
3,1800 Cristalino Anejo + GB,6,70.0,CL,35.0,€,189.00,31.50,Tequila,NaN,regular,MEXICO,7.501035e+12,17501035013480,anker
4,1800 Reposado,6,70.0,CL,38.0,€,97.50,16.25,Tequila,NaN,regular,MEXICO,7.501035e+12,17501035013221,anker


In [14]:
data = pd.DataFrame(columns = ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor'])
data = data.append(df1.loc[:, ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor']], ignore_index=True)
data = data.append(df2.loc[:, ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor']], ignore_index=True)
data = data.append(df3.loc[:, ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor']], ignore_index=True)
data = data.append(df4.loc[:, ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor']], ignore_index=True)
data = data.append(df5.loc[:, ['product', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle', 'price/case', 'offer', 'vendor']], ignore_index=True)
data['price/bottle'] = data.apply(lambda x: -1 if x['price/bottle'] == 'n.a.' else x['price/bottle'], axis=1)
data['price/case'] = data.apply(lambda x: -1 if x['price/case'] == 'n.a.' else x['price/case'], axis=1)
data.rename(columns={'price/bottle':'price/bottle (€)', 'price/case' : 'price/case (€)'}, inplace = True)
data = data.dropna(thresh=4)
data = data.fillna(0)
data['product'] = data['product'].astype('str') 
data = data.sort_values(by=['product'])
data = data.reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9096 entries, 0 to 9095
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           9096 non-null   object 
 1   volume            9096 non-null   object 
 2   unit              9096 non-null   object 
 3   alcohol (%)       9096 non-null   float64
 4   bottles/case      9096 non-null   float64
 5   price/bottle (€)  9096 non-null   float64
 6   price/case (€)    9096 non-null   float64
 7   offer             9096 non-null   object 
 8   vendor            9096 non-null   object 
dtypes: float64(4), object(5)
memory usage: 639.7+ KB


In [15]:
data.head()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor
0,1724 Tonic,20.0,CL,0.0,24.0,1.06,25.44,regular,anker
1,1800 Anejo,70.0,CL,38.0,6.0,18.25,109.50,Special,anker
2,1800 Anejo NRF,70.0,CL,38.0,6.0,18.75,112.50,regular,anker
3,1800 Cristalino Anejo + GB,70.0,CL,35.0,6.0,31.50,189.00,regular,anker
4,1800 Reposado,70.0,CL,38.0,6.0,16.25,97.50,regular,anker


In [16]:
data.to_excel("/content/drive/MyDrive/distilary_recommender_system/spirits_akay_refined.xlsx", index=False)  

## RAPIDFUZZ

In [17]:
!pip install -q rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.1 MB/s eta 0:00:00


In [18]:
from rapidfuzz import process, fuzz
# search_str = input("Which brand are you looking for? :   ")
search_str = "2019 Reserve Rouge"
most_similar = process.extract(search_str, data['product'], scorer=fuzz.WRatio, score_cutoff=40)
array = []

for i in most_similar:
  array.append(i[0])
# print(array)
# print(most_similar)
data.loc[data['product'].isin(array)].where(data['price/bottle (€)']>0).sort_values(by=['price/bottle (€)']).dropna()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor
14,2019 Il Medaglione Primitivo IGT,75.0,CL,13.5,6.0,4.3,25.8,regular,Bremer
59,A.H. Riise XO Ambre d'Or Reserve + GB,70.0,CL,42.0,6.0,24.8,148.8,regular,anker
51,A.H. Riise Ambre d'Or Reserve + 2 Glasses,70.0,CL,42.0,6.0,35.5,213.0,Special,anker


# Recommeder system

In [19]:
new_df = pd.read_excel('/content/drive/MyDrive/distilary_recommender_system/spirits_akay_refined.xlsx')
new_df.head()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker
1,1800 Anejo,70,CL,38.0,6,18.25,109.50,Special,anker
2,1800 Anejo NRF,70,CL,38.0,6,18.75,112.50,regular,anker
3,1800 Cristalino Anejo + GB,70,CL,35.0,6,31.50,189.00,regular,anker
4,1800 Reposado,70,CL,38.0,6,16.25,97.50,regular,anker


In [20]:
# Converting to lower case
new_df['tags'] = new_df['product'] + ' ' + new_df['volume'].astype(str) + ' CL'

In [ ]:
# Converting to lower case
# new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
# Converting to lower case
# new_df['product_lower'] = new_df['product'].apply(lambda x:x.lower())

In [21]:
new_df

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor,tags
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker,1724 Tonic 20 CL
1,1800 Anejo,70,CL,38.0,6,18.25,109.50,Special,anker,1800 Anejo 70 CL
2,1800 Anejo NRF,70,CL,38.0,6,18.75,112.50,regular,anker,1800 Anejo NRF 70 CL
3,1800 Cristalino Anejo + GB,70,CL,35.0,6,31.50,189.00,regular,anker,1800 Cristalino Anejo + GB 70 CL
4,1800 Reposado,70,CL,38.0,6,16.25,97.50,regular,anker,1800 Reposado 70 CL
...,...,...,...,...,...,...,...,...,...,...
9091,de Kuyper Sour Rhubarb,70,CL,15.0,6,5.90,35.40,regular,Bremer,de Kuyper Sour Rhubarb 70 CL
9092,de Kuyper Triple Sec,70,CL,40.0,6,5.15,30.90,regular,Bremer,de Kuyper Triple Sec 70 CL
9093,t Gilliske,70,CL,14.5,6,10.75,64.50,regular,anker,t Gilliske 70 CL
9094,t Nipperke,35,CL,14.5,12,5.20,62.40,regular,anker,t Nipperke 35 CL


In [ ]:
cols = ['tags', 'volume', 'unit', 'alcohol (%)', 'bottles/case', 'price/bottle (€)', 'price/case (€)', 'offer', 'vendor']
temp_df = new_df.iloc[:,:]
temp_df = temp_df.reindex(columns=cols)
temp_df.rename(columns = {'tags':'product'}, inplace=True)
temp_df = pd.concat([new_df, temp_df])
# temp_df.rename(columns = {'product':'product_list'}, inplace=True)
temp_df = temp_df.drop('tags', axis = 1)
temp_df

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker
1,1800 Anejo,70,CL,38.0,6,18.25,109.50,Special,anker
2,1800 Anejo NRF,70,CL,38.0,6,18.75,112.50,regular,anker
3,1800 Cristalino Anejo + GB,70,CL,35.0,6,31.50,189.00,regular,anker
4,1800 Reposado,70,CL,38.0,6,16.25,97.50,regular,anker
...,...,...,...,...,...,...,...,...,...
9091,de Kuyper Sour Rhubarb 70 CL,70,CL,15.0,6,5.90,35.40,regular,Bremer
9092,de Kuyper Triple Sec 70 CL,70,CL,40.0,6,5.15,30.90,regular,Bremer
9093,t Gilliske 70 CL,70,CL,14.5,6,10.75,64.50,regular,anker
9094,t Nipperke 35 CL,35,CL,14.5,12,5.20,62.40,regular,anker


In [ ]:
new_df.head()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor,tags
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker,1724 Tonic 20 CL
1,1800 Anejo,70,CL,38.0,6,18.25,109.50,Special,anker,1800 Anejo 70 CL
2,1800 Anejo NRF,70,CL,38.0,6,18.75,112.50,regular,anker,1800 Anejo NRF 70 CL
3,1800 Cristalino Anejo + GB,70,CL,35.0,6,31.50,189.00,regular,anker,1800 Cristalino Anejo + GB 70 CL
4,1800 Reposado,70,CL,38.0,6,16.25,97.50,regular,anker,1800 Reposado 70 CL


In [ ]:
temp_df['product']

0                         1724 Tonic
1                         1800 Anejo
2                     1800 Anejo NRF
3         1800 Cristalino Anejo + GB
4                      1800 Reposado
                    ...             
9091    de Kuyper Sour Rhubarb 70 CL
9092      de Kuyper Triple Sec 70 CL
9093                t Gilliske 70 CL
9094                t Nipperke 35 CL
9095                t Nipperke 70 CL
Name: product, Length: 18192, dtype: object

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [23]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity.shape

(9096, 9096)

In [25]:
similarities = pd.DataFrame(similarity, columns=new_df['tags'], index=new_df['tags']).reset_index()
similarities.head()

tags,tags,1724 Tonic 20 CL,1800 Anejo 70 CL,1800 Anejo NRF 70 CL,1800 Cristalino Anejo + GB 70 CL,1800 Reposado 70 CL,1800 Tequila Silver 70 CL,2015 Baron de Ley Gran Reserva 75 CL,2017 Baron de Ley Reserva 75 CL,2017 Baron de Ley Reserva 150 CL,...,de Kuyper Creme de Bananes 70 CL,de Kuyper Creme de Cacao weiß 70 CL,de Kuyper Curacao Blue 70 CL,de Kuyper Grenadine 70 CL,de Kuyper Melon 70 CL,de Kuyper Sour Rhubarb 70 CL,de Kuyper Triple Sec 70 CL,t Gilliske 70 CL,t Nipperke 35 CL,t Nipperke 70 CL
0,1724 Tonic 20 CL,1.000000,0.250000,0.223607,0.204124,0.250000,0.223607,0.188982,0.204124,0.204124,...,0.223607,0.204124,0.223607,0.250000,0.250000,0.223607,0.223607,0.288675,0.288675,0.288675
1,1800 Anejo 70 CL,0.250000,1.000000,0.894427,0.816497,0.750000,0.670820,0.188982,0.204124,0.204124,...,0.447214,0.408248,0.447214,0.500000,0.500000,0.447214,0.447214,0.577350,0.288675,0.577350
2,1800 Anejo NRF 70 CL,0.223607,0.894427,1.000000,0.730297,0.670820,0.600000,0.169031,0.182574,0.182574,...,0.400000,0.365148,0.400000,0.447214,0.447214,0.400000,0.400000,0.516398,0.258199,0.516398
3,1800 Cristalino Anejo + GB 70 CL,0.204124,0.816497,0.730297,1.000000,0.612372,0.547723,0.154303,0.166667,0.166667,...,0.365148,0.333333,0.365148,0.408248,0.408248,0.365148,0.365148,0.471405,0.235702,0.471405
4,1800 Reposado 70 CL,0.250000,0.750000,0.670820,0.612372,1.000000,0.670820,0.188982,0.204124,0.204124,...,0.447214,0.408248,0.447214,0.500000,0.500000,0.447214,0.447214,0.577350,0.288675,0.577350


In [26]:
new_df

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor,tags
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker,1724 Tonic 20 CL
1,1800 Anejo,70,CL,38.0,6,18.25,109.50,Special,anker,1800 Anejo 70 CL
2,1800 Anejo NRF,70,CL,38.0,6,18.75,112.50,regular,anker,1800 Anejo NRF 70 CL
3,1800 Cristalino Anejo + GB,70,CL,35.0,6,31.50,189.00,regular,anker,1800 Cristalino Anejo + GB 70 CL
4,1800 Reposado,70,CL,38.0,6,16.25,97.50,regular,anker,1800 Reposado 70 CL
...,...,...,...,...,...,...,...,...,...,...
9091,de Kuyper Sour Rhubarb,70,CL,15.0,6,5.90,35.40,regular,Bremer,de Kuyper Sour Rhubarb 70 CL
9092,de Kuyper Triple Sec,70,CL,40.0,6,5.15,30.90,regular,Bremer,de Kuyper Triple Sec 70 CL
9093,t Gilliske,70,CL,14.5,6,10.75,64.50,regular,anker,t Gilliske 70 CL
9094,t Nipperke,35,CL,14.5,12,5.20,62.40,regular,anker,t Nipperke 35 CL


In [30]:
result_df = pd.DataFrame(columns = new_df.columns)
def recommend(product):
    try:
      index = new_df[new_df['product'] == product.lower()].index[0]
      print('from tags')
    except IndexError:
      most_similar = process.extract(product, new_df['tags'], scorer=fuzz.WRatio, limit=1)
      index = most_similar[0][2]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key = lambda x: x[1])
    for i in distances[0:5]:
        if i[0]>=9096:
          result_df.loc[len(result_df)] = new_df.iloc[i[0]-9096]
        else:
          result_df.loc[len(result_df)] = new_df.iloc[i[0]]
        # print(new_df.iloc[i[0]])
    # 
recommend('1724 Tonic')
result_df = result_df.drop_duplicates()
result_df.iloc[:,:]
# result_df.iloc[:,:].sort_values(by=['price/bottle (€)'])
# result_df.iloc[:,:].where(result_df['price/case (€)']>0).sort_values(by=['price/bottle (€)']).dropna()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer,vendor,tags
0,1724 Tonic,20,CL,0.0,24,1.06,25.44,regular,anker,1724 Tonic 20 CL
1,Vin Tonic,20,CL,5.7,24,0.79,18.96,regular,Bremer,Vin Tonic 20 CL
2,Fentimans Tonic Water,20,CL,0.0,24,0.9,21.6,regular,anker,Fentimans Tonic Water 20 CL
3,Miracle 1638 tonic,20,CL,0.0,12,6.5,78.0,regular,anker,Miracle 1638 tonic 20 CL
4,Thomas Henry Tonic,20,CL,0.0,24,0.77,18.48,regular,anker,Thomas Henry Tonic 20 CL


In [31]:
result_df = pd.DataFrame(columns = new_df.columns[:-2])
recommend('absolut 100')
result_df.iloc[:,:].where(result_df['price/case (€)']>0).sort_values(by=['price/bottle (€)']).dropna()

,product,volume,unit,alcohol (%),bottles/case,price/bottle (€),price/case (€),offer
4,Szene Coconut,100,CL,21.0,6,2.99,17.94,regular
2,Finlandia Coconut,100,CL,37.5,12,5.95,71.4,Special
3,Finlandia Coconut,100,CL,37.5,12,6.5,78.0,regular
0,3 Kilos Coconut,100,CL,30.0,6,18.5,111.0,Special
1,3 Kilos Gold,100,CL,40.0,6,19.95,119.7,Special


In [33]:
import pickle
pickle.dump(new_df,open('/content/alcohol_list.pkl','wb'))
pickle.dump(similarity,open('/content/similarity.pkl','wb'))

In [34]:
import pickle
pickle.dump(new_df,open('/content/drive/MyDrive/distilary_recommender_system/alcohol_list.pkl','wb'))

In [35]:
pickle.dump(similarity,open('/content/drive/MyDrive/distilary_recommender_system/similarity.pkl','wb'))